In [357]:
import pandas as pd
import numpy as np
from fancyimpute import KNN
from sklearn.impute import KNNImputer
from sklearn import preprocessing 
import copy
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV,cross_val_predict
import sklearn.ensemble
import sklearn.model_selection
from sklearn import tree
import eazeml as ez
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [316]:
# !ls
# !pip install eazeml

In [557]:
train = pd.read_csv('Train_aBjfeNk.csv')
test = pd.read_csv('Test_LqhgPWU.csv')

In [558]:
idx = list(set(test['ID'].tolist()) & set(train['ID'].tolist()))
index = []
for i in range(len(test)):
    if test['ID'][i] in idx:
        index.append(i)

test_ = copy.deepcopy(test)
test_ = test_.drop(index)

In [559]:
def clean(train):
    del train['ID']
#     del train['Profession']
#     del train['Var_1']
    
#     del train['Segmentation']
    train['Gender'] = train['Gender'].map({'Male':0, 'Female':1})
    train['Ever_Married'] = train['Ever_Married'].map({'No':0, 'Yes':1})
    train['Graduated'] = train['Graduated'].map({'No':0, 'Yes':1})
    train['Spending_Score'] = train['Spending_Score'].map({'Low':0, 'Average':1, 'High':2})
    train['Var_1'] = train['Var_1'].map({'Cat_1':0, 'Cat_2':1, 'Cat_3':2, 'Cat_4':3, 'Cat_5':4, 'Cat_6':5,'Cat_7':6})
    
#     train['Ever_Married'] = train['Ever_Married'].fillna(train['Ever_Married'].mode().iloc[0])
#     train['Graduated'] = train['Graduated'].fillna(train['Graduated'].mode().iloc[0])
#     train['Var_1'] = train['Var_1'].fillna(train['Var_1'].mode().iloc[0])
#     train['Profession'] = train['Profession'].fillna(train['Profession'].mode().iloc[0])
#     train['Work_Experience'] = train['Work_Experience'].fillna(train['Work_Experience'].mean())
#     train['Family_Size'] = train['Family_Size'].fillna(train['Family_Size'].mean())

    x = pd.get_dummies(train)

    my_imputer = KNNImputer(n_neighbors=3)
    my = my_imputer.fit(x)
    clean_df = pd.DataFrame(my.transform(x))

    clean_df.columns = x.columns
    clean_df.index = x.index

    return clean_df

In [560]:
train_1 = copy.deepcopy(train)
test_1 = copy.deepcopy(test_)
del train_1['Segmentation']
train_df = clean(train_1)
test_df = clean(test_1)

In [561]:
X_train = train_df
Y_train = train['Segmentation']
X_test = test_df

In [562]:
def study_(X,y):
    
    def objective(trial):

        n_estimators = trial.suggest_int('n_estimators', 1, 30)
        max_depth = int(trial.suggest_int('max_depth', 1, 32, log=True))

        clf = XGBClassifier(
            n_estimators=n_estimators, max_depth=max_depth)

        return sklearn.model_selection.cross_val_score(
            clf, X, y, n_jobs=-1, cv=5).mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective,n_trials=50)

    trial = study.best_trial

    print('Accuracy: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))
    return trial.params

In [563]:
best_p = study_(X_train,Y_train)
rfr = XGBClassifier(max_depth=best_p["max_depth"], n_estimators=best_p["n_estimators"],random_state=42)
      
score = cross_val_score(rfr, X_train, Y_train, cv=5)
print(score)
      
model = rfr.fit(X_train, Y_train)
result = model.predict(X_test)

[I 2020-08-02 19:15:22,297] Finished trial#0 with value: 0.4812840374558939 with parameters: {'n_estimators': 26, 'max_depth': 23}. Best is trial#0 with value: 0.4812840374558939.
[I 2020-08-02 19:15:26,277] Finished trial#1 with value: 0.4837635813722304 with parameters: {'n_estimators': 6, 'max_depth': 31}. Best is trial#1 with value: 0.4837635813722304.
[I 2020-08-02 19:15:28,343] Finished trial#2 with value: 0.49070409183131786 with parameters: {'n_estimators': 5, 'max_depth': 15}. Best is trial#2 with value: 0.49070409183131786.
[I 2020-08-02 19:15:29,172] Finished trial#3 with value: 0.5013641486343533 with parameters: {'n_estimators': 17, 'max_depth': 1}. Best is trial#3 with value: 0.5013641486343533.
[I 2020-08-02 19:15:30,022] Finished trial#4 with value: 0.5228081779777228 with parameters: {'n_estimators': 11, 'max_depth': 3}. Best is trial#4 with value: 0.5228081779777228.
[I 2020-08-02 19:15:31,085] Finished trial#5 with value: 0.5203277890067612 with parameters: {'n_estim

[I 2020-08-02 19:17:00,443] Finished trial#45 with value: 0.5231780046109253 with parameters: {'n_estimators': 22, 'max_depth': 2}. Best is trial#24 with value: 0.5366888916033068.
[I 2020-08-02 19:17:03,220] Finished trial#46 with value: 0.5343354912955532 with parameters: {'n_estimators': 26, 'max_depth': 5}. Best is trial#24 with value: 0.5366888916033068.
[I 2020-08-02 19:17:04,518] Finished trial#47 with value: 0.5280129462368566 with parameters: {'n_estimators': 16, 'max_depth': 3}. Best is trial#24 with value: 0.5366888916033068.
[I 2020-08-02 19:17:06,186] Finished trial#48 with value: 0.5262774345063459 with parameters: {'n_estimators': 28, 'max_depth': 2}. Best is trial#24 with value: 0.5366888916033068.
[I 2020-08-02 19:17:09,420] Finished trial#49 with value: 0.5356975657049177 with parameters: {'n_estimators': 23, 'max_depth': 4}. Best is trial#24 with value: 0.5366888916033068.


Accuracy: 0.5366888916033068
Best hyperparameters: {'n_estimators': 22, 'max_depth': 4}
[0.54089219 0.53221809 0.53097893 0.5399876  0.53936764]


In [564]:
data = {'ID': test_['ID'].tolist(),'Segmentation': result.tolist()}
final_ = pd.DataFrame(data)

In [ ]:
idx = list(set(train['ID'].tolist()) - set(test['ID'].tolist()))
index = []
for i in range(len(train)):
    if train['ID'][i] in idx:
        index.append(i)

inter = copy.deepcopy(train)
inter = inter.drop(index)

In [550]:
inter = inter[['ID','Segmentation']]

In [551]:
final = inter+final_

In [556]:
final_df = pd.concat([inter,final_])

In [473]:
# segment = []
# for i in range(len(final)):
#     c = 0
#     for j in range(len(train)):
#         if final['ID'][i] == train['ID'][j]:
#             c = 1
#             segment.append(train['Segmentation'][j])
#             break
#     if c == 0:
#         segment.append(final['Segmentation'][i])
# final['Segmentation'] = segment

In [476]:
final_df.to_csv('sol.csv',index = False)